# Test Library

# Confirm Library Works

In [3]:
from libraries.SQLite_Database import SQLite_Database
db_path = 'database/data_science_application.db'
file_path = 'static/addressbgt.csv'
table_name = 'business_metadata'

In [43]:
db = SQLite_Database(db_path)

# Verify the schema before insertion
print(db.fetch_query("SELECT * FROM business_metadata"))

   business_glossary_term_id business_glossary_term  \
0                         21              'Address'   

  business_glossary_definition  
0           'Customer Address'  


In [41]:
table_name = 'business_metadata'
columns = ['business_glossary_term_id']
new_values = [21]  # Ensure this is an integer if that's the column type

db.update_table(table_name, columns, new_values)

Table 'business_metadata' updated with new values: {'business_glossary_term_id': 21}.


In [84]:
from libraries.SQLite_Database import SQLite_Database
db_path = 'database/data_science_application.db'
db = SQLite_Database(db_path)
db.get_sqlite_metadata()

,Table Name,Column Name,Data Type
0,business_metadata,business_glossary_term_id,INTEGER
1,business_metadata,business_glossary_term,TEXT
2,business_metadata,business_glossary_definition,TEXT
3,diamonds,Unnamed: 0,INTEGER
4,diamonds,carat,REAL
5,diamonds,cut,TEXT
6,diamonds,color,TEXT
7,diamonds,clarity,TEXT
8,diamonds,depth,REAL
9,diamonds,table,REAL


In [80]:
print(dir(SQLite_Database))
print(dir(SQLite_Database))
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'close', 'connect', 'create_sqlite_database', 'create_table_from_query', 'fetch_query', 'get_sqlite_metadata', 'insert_csv_to_db', 'insert_csv_to_table', 'update_table']

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'close', 'connect', 'create_sqlite_database', 'create_table_from_query', 'fetch_query', 'get_sqlite_metadata', 'insert_csv_to_db', 'insert_csv_to_table', 'update_table']


In [77]:
Code: 
from libraries.SQLite_Database import SQLite_Database
db_path = 'database/data_science_application.db'
db = SQLite_Database(db_path)
db.get_tables_metadata()

Error:
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[77], line 4
      2 db_path = 'database/data_science_application.db'
      3 db = SQLite_Database(db_path)
----> 4 db.get_tables_metadata()

AttributeError: 'SQLite_Database' object has no attribute 'get_tables_metadata'

libraries.SQLite_Database code 

import sqlite3
import pandas as pd
from flask import g

class SQLite_Database:
    def __init__(self, db_path):
        self.db_path = db_path

    def connect(self):
        """Establish a connection to the SQLite database."""
        if 'conn' not in g:
            g.conn = sqlite3.connect(self.db_path)
        return g.conn

    def close(self):
        """Close the connection to the SQLite database."""
        conn = g.pop('conn', None)
        if conn:
            conn.close()

    def insert_dataframe_to_db(self, df, table_name):
        """
        Inserts a pandas DataFrame into the SQLite database.

        Parameters:
        df (pandas.DataFrame): The DataFrame containing the CSV data.
        table_name (str): The name of the table to insert data into.
        """
        conn = self.connect()
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        conn.commit()

    def fetch_table(self, table_name):
        """
        Fetches all records from a specific table.

        Parameters:
        table_name (str): Name of the table to fetch records from.

        Returns:
        pd.DataFrame: DataFrame containing all records from the table.
        """
        conn = self.connect()
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, conn)
        return df

    def get_tables_metadata(self):
        """
        Retrieves metadata for all tables in the SQLite database.

        Returns:
        pd.DataFrame: DataFrame containing table names, column names, and data types.
        """
        conn = self.connect()
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        tables_df = pd.read_sql_query(query, conn)
        metadata = []

        for table in tables_df['name']:
            columns_query = f'PRAGMA table_info("{table}");'
            columns_df = pd.read_sql_query(columns_query, conn)
            for _, row in columns_df.iterrows():
                metadata.append((table, row['name'], row['type']))

        metadata_df = pd.DataFrame(metadata, columns=['Table Name', 'Column Name', 'Data Type'])
        return metadata_df

    def execute_query(self, query):
        """
        Executes a raw SQL query and returns the result as a DataFrame.

        Parameters:
        query (str): The SQL query to execute.

        Returns:
        pd.DataFrame: The result of the query.
        """
        conn = self.connect()
        df = pd.read_sql_query(query, conn)
        return df

    def insert_record(self, table_name, columns, values):
        """
        Inserts a single record into a specific table.

        Parameters:
        table_name (str): Name of the table to insert the record into.
        columns (list): List of column names.
        values (list): List of values to insert.
        """
        conn = self.connect()
        placeholders = ', '.join(['?'] * len(values))
        query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"
        cursor = conn.cursor()
        cursor.execute(query, values)
        conn.commit()

    def update_record(self, table_name, columns, values, condition):
        """
        Updates a single record in a specific table.

        Parameters:
        table_name (str): Name of the table to update the record in.
        columns (list): List of column names to update.
        values (list): List of new values.
        condition (str): SQL condition to identify the record(s) to update.
        """
        conn = self.connect()
        set_clause = ', '.join([f"{col} = ?" for col in columns])
        query = f"UPDATE {table_name} SET {set_clause} WHERE {condition}"
        cursor = conn.cursor()
        cursor.execute(query, values)
        conn.commit()

    def delete_record(self, table_name, condition):
        """
        Deletes a record from a specific table.

        Parameters:
        table_name (str): Name of the table to delete the record from.
        condition (str): SQL condition to identify the record(s) to delete.
        """
        conn = self.connect()
        query = f"DELETE FROM {table_name} WHERE {condition}"
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()

    def clear_table(self, table_name):
        """
        Clears all records from a specific table.

        Parameters:
        table_name (str): Name of the table to clear.
        """
        conn = self.connect()
        query = f"DELETE FROM {table_name}"
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()


AttributeError: 'SQLite_Database' object has no attribute 'get_tables_metadata'